# Installing the NAG library and running this notebook

This notebook depends on the NAG library for Python to run. Please read the instructions in the [Readme.md](https://github.com/numericalalgorithmsgroup/NAGPythonExamples/blob/master/local_optimization/Readme.md#install) file to download, install and obtain a licence for the library.

Instruction on how to run the notebook can be found [here](https://github.com/numericalalgorithmsgroup/NAGPythonExamples/blob/master/local_optimization/Readme.md#jupyter).

# Minimizing the generalized Rosenbrock function

Suppose that we wish to minimize the ‘generalized Rosenbrock’ function using
bound constrained optimization.

Interfaces to the NAG Library are provided in the ``naginterfaces.library``
subpackage

One can see from the HTML documentation at https://www.nag.com/numeric/py/nagdoc_latest/naginterfaces.library.html that the relevant algorithmic submodule for (local) optimization is ``opt``.

Studying the `opt` Functionality Index confirms that the relevant optimization solver to call is
``bounds_quasi_func_easy``. The HTML documentation for this solver is at https://www.nag.com/numeric/py/nagdoc_latest/naginterfaces.library.opt.bounds_quasi_func_easy.html.

The optimization solver may be imported directly if desired

In [1]:
from naginterfaces.library.opt import bounds_quasi_func_easy

Now define the optimization problem: first, the objective function for the
generalized Rosenbrock problem. From the documented signature of the NAG routine

In [2]:
help(bounds_quasi_func_easy)

Help on function bounds_quasi_func_easy in module naginterfaces.library.opt:

bounds_quasi_func_easy(ibound, funct1, bl, bu, x, liw=None, lw=None, data=None)
    Bound constrained minimum, quasi-Newton algorithm, using function
    values only (easy-to-use).
    
    ``bounds_quasi_func_easy`` is an easy-to-use quasi-Newton algorithm
    for finding a minimum of a function F(x_1,x_2,...,x_n), subject to
    fixed upper and lower bounds of the independent variables
    x_1,x_2,...,x_n, using function values only.
    
    It is intended for functions which are continuous and which have
    continuous first and second derivatives (although it will usually
    work even if the derivatives have occasional discontinuities).
    
    For full information please refer to the NAG Library document for
    e04jy
    
    https://www.nag.com/numeric/nl/nagdoc_27.1/flhtml/e04/e04jyf.html
    
    Parameters
    ----------
    ibound : int
        Indicates whether the facility for dealing with bou

we can infer that parameter ``funct1`` may be specified as a
lambda expression in our case, where we do not have any communication data
(``data``) to pass to the function

In [3]:
rosen = lambda x: (sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1.0-x[:-1])**2.0))

Here is a contour plot of the Rosenbrock function in 2D

In [4]:
# Select the display backend for Jupyter:
%matplotlib nbagg

In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np

delta = 0.05
x = np.arange(-3., 3., delta)
y = np.arange(-3., 3, delta)
X, Y = np.meshgrid(x, y)
Z = np.empty((len(X), len(Y)))
for i, x_i in enumerate(x):
    for j, y_j in enumerate(y):
        Z[j, i] = rosen(np.array([x_i, y_j]))

fig = plt.figure()
ax = Axes3D(fig, auto_add_to_figure=False)
ax.grid(False)
ax.plot_wireframe(X, Y, Z, color='red', linewidths=0.4)
ax.contour(X, Y, Z, levels=[5, 25, 50, 100, 250, 500, 1000, 1500, 2000, 2500], offset=-3000.0, cmap=cm.jet)
ax.set_title('The 2D Rosenbrock Function')
ax.set_xlabel(r'$\mathit{x}$')
ax.set_ylabel(r'$\mathit{y}$')
ax.set_zlim3d(-1.0, 10000.0)
ax.azim = -20
ax.elev = 20
fig.add_axes(ax)
plt.show()

<IPython.core.display.Javascript object>

Finding the minimum of the 2D Rosenbrock function is relatively straightforward so we're going to find a minimum of the 10D Rosenbrock function (we can't plot that!).

First we define an initial guess for the optimization. In the
``naginterfaces.library`` subpackage input array data may be supplied in any
‘array-like’ container, as noted above in the ``float, array-like, shape (n)``
specification for argument ``x``. For our one-dimensional ``x``, this means
that any sequence of data will be a suitable container, as will a
``numpy.ndarray``.
(In functions taking multi-dimensional data, nested sequences and again
instances of ``numpy.ndarray`` are valid.) Furthermore, the shape (length) of
the ``x`` we supply determines the (inferred) value of ``n`` for the problem.

Our chosen start point is $(0., 0., ..., 0., 0.)$ and thus any of the
following may be used to supply the ‘array-like’ vector ``x``:

- as a ``list``


In [6]:
x = [0.]*10

- as a ``tuple``

In [7]:
x = (0., 0., 0., 0., 0., 0., 0., 0., 0., 0.)

- as an ``ndarray``


In [8]:
x = np.array([0.]*10)

Now define box bounds for the problem

In [9]:
n = len(x)
bl, bu = [0.0]*n, [2.0]*n
ibound = 0

Minimize the problem

In [10]:
opt_soln = bounds_quasi_func_easy(ibound, rosen, bl, bu, x)

Arguments returned by the NAG routine can be accessed as named fields of the return tuple.

Display the results

In [11]:
print('Function value at lowest point found is {:.5f}.'.format(opt_soln.f))
print('The corresponding x is (' + ', '.join(['{:.4f}'] * n).format(*opt_soln.x) + ').')

Function value at lowest point found is 0.00000.
The corresponding x is (1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000).
